# Import and Explore Data in SQLAlchemy

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [2]:
# connect to local database - works if jupyter notebook is in same folder as db otherwise specify a path
engine = create_engine("sqlite:///mta_data.db")

In [12]:
# example of deprecation error
engine.table_names()

/var/folders/_s/x12bz48x21196st1tv_g16_00000gn/T/ipykernel_16280/2782894312.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['mta_data']

In [3]:
insp = inspect(engine)
print(insp.get_table_names())

['mta_data']


In [4]:
# To find count of all rows wrapping * in the count function can add alias, mta_data is a data frame
mta_data = pd.read_sql('SELECT COUNT(*) FROM mta_data;', engine)
mta_data

,COUNT(*)
0,7535013


In [5]:
# To change column header, use AS - this creates an alias
# DISTINCT  will find the unique values in the rows
mta_data = pd.read_sql('SELECT COUNT(DISTINCT STATION) AS TOTAL_STATIONS FROM mta_data;', engine)
mta_data

,TOTAL_STATIONS
0,379


In [6]:
# ? This doesn't really mean anything, how can I chain C/A, SCP, UNIT and STATION as the primary key to see total
# number of turnstiles
mta_data = pd.read_sql('SELECT COUNT(DISTINCT SCP) FROM mta_data;', engine)
mta_data

,COUNT(DISTINCT SCP)
0,230


In [7]:
mta_data = pd.read_sql('SELECT DISTINCT STATION AS STATION_LIST FROM mta_data;', engine)
mta_data

,STATION_LIST
0,59 ST
1,5 AV/59 ST
2,57 ST-7 AV
3,49 ST
4,TIMES SQ-42 ST
...,...
374,EASTCHSTER/DYRE
375,ST. GEORGE
376,TOMPKINSVILLE
377,RIT-MANHATTAN


In [8]:
# High sums and averages on entries and exits seems implusable, high min max indicates there may be anomolies in the data, seeing zeros in this query needs futher investigation
# Note - agregations are done in the SELECT clause
mta_data = pd.read_sql('SELECT SUM(ENTRIES), SUM(EXITS), AVG(ENTRIES), AVG(EXITS), MIN(ENTRIES), MAX(ENTRIES), MIN(EXITS), MAX(EXITS) FROM mta_data;', engine)
mta_data

,SUM(ENTRIES),SUM(EXITS),AVG(ENTRIES),AVG(EXITS),MIN(ENTRIES),MAX(ENTRIES),MIN(EXITS),MAX(EXITS)
0,315919635145088,253504919206566,4.192689e+07,3.364359e+07,0,2147432317,0,2133796536


In [9]:
# Query the number of rows of data collected by station 
mta_data = pd.read_sql('SELECT STATION, COUNT(*), COUNT(ENTRIES), COUNT(EXITS) FROM mta_data GROUP BY STATION;', engine)
mta_data

,STATION,COUNT(*),COUNT(ENTRIES),COUNT(EXITS)
0,1 AV,36541,36541,36541
1,103 ST,22683,22683,22683
2,103 ST-CORONA,13588,13588,13588
3,104 ST,13605,13605,13605
4,110 ST,9052,9052,9052
...,...,...,...,...
374,WOODLAWN,7545,7545,7545
375,WORLD TRADE CTR,27155,27155,27155
376,WTC-CORTLANDT,52850,52850,52850
377,YORK ST,4562,4562,4562


In [10]:
mta_data = pd.read_sql('SELECT * FROM mta_data WHERE EXITS IS NOT NULL LIMIT 10;', engine)
mta_data

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,00:00:00,REGULAR,7625998,2609113
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,04:00:00,REGULAR,7626014,2609117
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,08:00:00,REGULAR,7626024,2609135
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,12:00:00,REGULAR,7626086,2609191
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,16:00:00,REGULAR,7626206,2609221
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,20:00:00,REGULAR,7626348,2609238
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2021,00:00:00,REGULAR,7626446,2609254
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2021,04:00:00,REGULAR,7626452,2609259
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2021,08:00:00,REGULAR,7626464,2609272
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2021,12:00:00,REGULAR,7626508,2609309


In [16]:
# STATIONS capturing the most lines of data could indicate busiest station
mta_data = pd.read_sql('SELECT STATION, COUNT(ENTRIES) FROM mta_data GROUP BY STATION HAVING COUNT(ENTRIES) > 80000 ORDER BY COUNT(ENTRIES) DESC ;', engine)
mta_data

,STATION,COUNT(ENTRIES)
0,34 ST-PENN STA,153853
1,FULTON ST,145039
2,GRD CNTRL-42 ST,134978
3,23 ST,110397
4,86 ST,98299
5,CANAL ST,87664
6,59 ST,84600
7,CHAMBERS ST,83237
8,34 ST-HERALD SQ,81746


In [18]:
# same query as above but this format is not saving the results in a df. In previous examples I might be overwriting the variables.
pd.read_sql('SELECT STATION, COUNT(ENTRIES) FROM mta_data GROUP BY STATION HAVING COUNT(ENTRIES) > 80000 ORDER BY COUNT(ENTRIES) DESC;', engine)

,STATION,COUNT(ENTRIES)
0,34 ST-PENN STA,153853
1,FULTON ST,145039
2,GRD CNTRL-42 ST,134978
3,23 ST,110397
4,86 ST,98299
5,CANAL ST,87664
6,59 ST,84600
7,CHAMBERS ST,83237
8,34 ST-HERALD SQ,81746


In [14]:
pd.read_sql('SELECT "C/A", UNIT, SCP, STATION, DESC, DIVISION FROM mta_data WHERE DESC NOT LIKE "REGULAR" AND DIVISION LIKE "BMT" LIMIT 20;', engine)

,C/A,UNIT,SCP,STATION,DESC,DIVISION
0,A060,R001,00-00-06,WHITEHALL S-FRY,RECOVR AUD,BMT
1,B014,R148,00-00-00,PARKSIDE AV,RECOVR AUD,BMT
2,B014,R148,00-00-01,PARKSIDE AV,RECOVR AUD,BMT
3,B014,R148,00-00-02,PARKSIDE AV,RECOVR AUD,BMT
4,B015,R098,01-00-00,CHURCH AV,RECOVR AUD,BMT
5,B015,R098,01-00-00,CHURCH AV,RECOVR AUD,BMT
6,B015,R098,01-00-00,CHURCH AV,RECOVR AUD,BMT
7,B015,R098,01-00-01,CHURCH AV,RECOVR AUD,BMT
8,B015,R098,01-00-01,CHURCH AV,RECOVR AUD,BMT
9,B015,R098,01-00-01,CHURCH AV,RECOVR AUD,BMT


In [15]:
pd.read_sql('SELECT * FROM mta_data WHERE DESC NOT LIKE "REGULAR";', engine)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A060,R001,00-00-06,WHITEHALL S-FRY,R1W,BMT,09/02/2021,05:00:00,RECOVR AUD,667307,1280229
1,B014,R148,00-00-00,PARKSIDE AV,BQ,BMT,08/31/2021,13:00:00,RECOVR AUD,11008572,18014220
2,B014,R148,00-00-01,PARKSIDE AV,BQ,BMT,08/31/2021,13:00:00,RECOVR AUD,16411910,9880877
3,B014,R148,00-00-02,PARKSIDE AV,BQ,BMT,08/31/2021,13:00:00,RECOVR AUD,15376905,9431942
4,B015,R098,01-00-00,CHURCH AV,BQ,BMT,08/28/2021,08:00:00,RECOVR AUD,3575959,2173597
...,...,...,...,...,...,...,...,...,...,...,...
32028,R635,R277,00-00-01,PRESIDENT ST,25,IRT,01/01/2021,23:00:00,RECOVR AUD,6204938,8045300
32029,R635,R277,00-00-02,PRESIDENT ST,25,IRT,01/01/2021,23:00:00,RECOVR AUD,68046,84708
32030,R636,R209,00-00-00,STERLING ST,25,IRT,12/26/2020,07:00:00,RECOVR AUD,11885815,5449073
32031,R636,R209,00-00-01,STERLING ST,25,IRT,12/26/2020,07:00:00,RECOVR AUD,5063722,3198251


In [29]:
# result indicates almost all stations have RECOVR AUD situations total number reporting is 348 out of 379 stations
pd.read_sql('SELECT * FROM mta_data WHERE DESC NOT LIKE "REGULAR" GROUP BY [STATION] HAVING COUNT(DESC) > 1 ORDER BY [STATION];', engine)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,H007A,R248,02-00-00,1 AV,L,BMT,08/24/2021,09:51:31,RECOVR AUD,335355,50458
1,R252,R180,00-00-00,103 ST,6,IRT,04/27/2021,21:00:00,RECOVR AUD,1004933,1000221
2,R529,R208,00-05-01,103 ST-CORONA,7,IRT,04/18/2021,00:00:00,RECOVR AUD,12114123,8255599
3,N137,R354,00-00-00,104 ST,A,IND,08/15/2021,00:00:00,RECOVR AUD,12329,10453
4,R254,R181,01-00-00,110 ST,6,IRT,08/14/2021,12:00:00,RECOVR AUD,4633924,2518678
...,...,...,...,...,...,...,...,...,...,...,...
343,R294,R052,00-00-00,WOODLAWN,4,IRT,07/06/2021,08:00:00,RECOVR AUD,8968868,7371491
344,N094,R029,01-03-02,WORLD TRADE CTR,ACE23,IND,08/29/2021,04:00:00,RECOVR AUD,5758035,4323430
345,R108,R305,02-00-00,WTC-CORTLANDT,1,IRT,06/14/2021,17:00:00,RECOVR AUD,101805,726868
346,N530,R301,00-00-00,YORK ST,F,IND,05/29/2021,17:00:00,RECOVR AUD,129245,289619
